In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Retrieving data with StatAPI

### Imports

In [ ]:
import datetime as dt
from collections import defaultdict
from functools import partial

import json
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as cols

import shapely

In [ ]:
from sentinelhub import (
    SentinelHubRequest, DataCollection, 
    SentinelHubStatistical, SentinelHubStatisticalDownloadClient,
    CRS, BBox, Geometry, bbox_to_dimensions, 
    parse_time, parse_time_interval, MimeType
)

## Some helper functions

Once StatAPI reaches maturity, these functions will be available in `sh-py`.

In [ ]:
def stat_to_df(stat_data):
    """ transform response from StatAPI into dataframe"""
    df_data = defaultdict(list)

    for single_data in stat_data['data']:
        df_data['interval_from'].append(parse_time(single_data['interval']['from']).date())
        df_data['interval_to'].append(parse_time(single_data['interval']['to']).date())
        
        for output_data in single_data['outputs'].keys():
            single_band = len(single_data['outputs'][output_data]['bands']) == 1
            for band_name, band_values in single_data['outputs'][output_data]['bands'].items():
                for stat_name, value in band_values['stats'].items():
                    col_name = f'{output_data}_{band_name}_{stat_name}'
                    if stat_name == 'percentiles':
                        for perc, perc_val in value.items():
                            perc_col_name = f'{col_name}_{perc}'
                            df_data[perc_col_name].append(perc_val)
                    else:
                        df_data[col_name].append(value)
    
    df = pd.DataFrame(df_data)
    # df = df.astype({c:'float' for c in df.columns if any(n in c for n in ['mean','min','max','stDev','percentil'])})
    return df


In [ ]:
def to_download_requests(gdf, data_folder=None):
    """ function to create StatAPI request per each geometry in geopandas geoDataFrame 
    
    :param: data_folder: specify a folder to cache the responses from SH service - particularly useful when testing
    """
    
    stat_requests = []
    for row in gdf.itertuples():
        req = SentinelHubStatistical(
            aggregation=aggregation, 
            calculations=calculations, 
            input_data=[input_data], 
            geometry=Geometry(row.geometry, crs=CRS(gdf.crs.to_epsg())),
            data_folder=data_folder
        )
        stat_requests.append(req)
    
    download_requests = [stat_request.download_list[0] for stat_request in stat_requests]
    
    if data_folder:
        for download_request in download_requests:
            download_request.save_response = True
    
    return download_requests

## Fetch statistics using sh-py + StatAPI

Evalscript to retrieve data:

In [ ]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B10", "B11", "B12", "CLM", "CLP", "dataMask"],
      units: "DN"
    }],
    output: [
      {
        id: "bands",
        bands: ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B10", "B11", "B12"],
        sampleType: "UINT16"
      },
      {
        id: "masks",
        bands: ["CLM"],
        sampleType: "UINT16"
      },
      {
        id: "indices",
        bands: ["NDVI", "NDVI_RE1", "NBSI", "CLP"],
        sampleType: "UINT16"
      },
      {
        id: "dataMask",
        bands: 1
      }]
  }
}

function evaluatePixel(samples) {
    // Normalised Difference Vegetation Index and variation
    let NDVI = index(samples.B08, samples.B04);
    let NDVI_RE1 = index(samples.B08, samples.B05);

    // Bare Soil Index 
    let NBSI = index((samples.B11 + samples.B04), (samples.B08 + samples.B02));
    
    // cloud probability normalized to 0..1
    let CLP = samples.CLP/255.0;
        
    const f = 5000;
    return {
        bands: [samples.B01, samples.B02, samples.B03, samples.B04, samples.B05, samples.B06, 
                samples.B07, samples.B08, samples.B8A, samples.B09, samples.B10, samples.B11, samples.B12],
        masks: [samples.CLM],
        indices: [toUINT(NDVI, f), toUINT(NDVI_RE1, f), toUINT(NBSI, f), toUINT(CLP, f)],
        dataMask: [samples.dataMask]
    };
}

function toUINT(product, constant){
  // Clamp the output to [-1, 10] and convert it to a UNIT16
  // value that can be converted back to float later.
  if (product < -1) {
    product = -1;
  } else if (product > 10) {
    product = 10;
  }
  return Math.round(product * constant) + constant;
}

function feature_ratio(band_a, band_b, constant){
  // Compute Band Ratio of the form A/B + constant.
  return (band_a / band_b) + constant;
}

"""

In [ ]:
bbox = BBox(((407670.0, 5124960.0), (412270.0, 5128960.0)), crs=CRS.UTM_33N)

In [ ]:
time_interval = ('2020-01-01','2020-12-31')

In [ ]:
resolution=(10,10)

In [ ]:
aggregation = SentinelHubStatistical.aggregation(
    evalscript=evalscript,
    time_interval=('2020-01-01','2021-01-01'),
    aggregation_interval='P1D',
    size=bbox_to_dimensions(bbox, 10))

In [ ]:
input_data = SentinelHubRequest.input_data(DataCollection.SENTINEL2_L1C, maxcc=1)

<div class="alert alert-block alert-info">
    We will request default statistics + [5,50,90]th percentiles for all outputs.

In [ ]:
calculations = {
    "default": {
      "statistics": {
        "default": {
          "percentiles": {
            "k": [5, 50, 95]
          }
        }
      }
    }
}

In [ ]:
stats = SentinelHubStatistical(
    aggregation=aggregation, 
    calculations=calculations, 
    input_data=[input_data], 
    bbox=bbox
)

In [ ]:
%%time
res = stats.get_data()[0]

<div class="alert alert-block alert-info">
Beware that with StatAPI the response can contain errors for particular intervals. (With FIS, whole response failed if single timestamp failed.) This is how erroneous data looks like:
</div>

```javascript
{'interval': {'from': '2020-12-21T00:00:00Z', 'to': '2020-12-22T00:00:00Z'}, 'error': {'type': 'EXECUTION_ERROR'}}
{'interval': {'from': '2020-12-26T00:00:00Z', 'to': '2020-12-27T00:00:00Z'}, 'error': {'type': 'EXECUTION_ERROR'}}
{'interval': {'from': '2020-12-31T00:00:00Z', 'to': '2021-01-01T00:00:00Z'}, 'error': {'type': 'EXECUTION_ERROR'}}
```

In [ ]:
res['data'][0]['interval']

In [ ]:
res['data'][2]['outputs']['indices']['bands']['NDVI']

<div class="alert alert-block alert-info">
Converting the response to `pandas` dataframe, we get:
</div>



In [ ]:
res_df = stat_to_df(res)

In [ ]:
res_df.head()

### Conversion function to back to "float"

<div class="alert alert-block alert-info">
As our evalscript is requesting outputs as unsigned integers, we want to transform statistics back:
</div>

In [ ]:
float_res_df = res_df.copy()

for index in ["NDVI", "NDVI_RE1", "NBSI", "CLP"]:
    for stat in ['mean', 'min', 'max', 'stDev', 'percentiles_5.0', 'percentiles_50.0', 'percentiles_95.0']:
        col = f'indices_{index}_{stat}'
        if stat == 'stDev':
            float_res_df[col] = float_res_df[col] / 5000.
        else:
            float_res_df[col] = (float_res_df[col] - 5000.) / 5000.

<div class="alert alert-block alert-info">
Note: statistics for S-2 bands are still in DNs, so should be divided by 1.e4 to get to statistics for reflectances.
</div>

In [ ]:
float_res_df.head()

## Run on all geometries from geodataframe

<div class="alert alert-block alert-info">
    To showcase how to run this on a large(r) number of geometries, using `sh-py` to do the multithreading etc., we've constructed a geoDataFrame with four polygons somewhere in Slovenia, over four different land covers: water, forest, agricultural fields and urban fabric.

In [ ]:
gdf = gpd.read_file('../data/statapi_test.geojson')

In [ ]:
gdf.head()

<div class="alert alert-block alert-info">
    We will convert the geometries to UTM_33N coordinate reference system, as the Sentinel-2 tiles in Slovenia are originally in this UTM, but we could do the requests in WGS as well.

In [ ]:
gdf_utm = gdf.to_crs(epsg=32633)

In [ ]:
download_requests = to_download_requests(gdf_utm)

In [ ]:
client = SentinelHubStatisticalDownloadClient()
stat_data = client.download(download_requests)

In [ ]:
len(stat_data)

<div class="alert alert-block alert-info">
    We got back 4 responses, ordered in the same way as the rows in the `gdf_utm` are, so we can construct a full dataframe (adding land cover type) to the responses, and transforming the statistics for indices:


In [ ]:
indices = list(stat_data[0]['data'][0]['outputs']['indices']['bands'].keys())
statistics = ['min', 'max', 'mean', 'stDev']

dfs = []
for idx, stat in enumerate(stat_data):
    df = stat_to_df(stat)
    
    for index in indices:
        for stat in statistics:
            col = f'indices_{index}_{stat}'
            if stat == 'stDev':
                df[col] = df[col] / 5000.
            else:
                df[col] = (df[col] - 5000.) / 5000.
    
    df['type'] = gdf_utm.iloc[idx].type
    dfs.append(df)

data = pd.concat(dfs)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
data.set_index('interval_from').groupby('type')['indices_NDVI_mean'].plot(ax=ax, legend=True);

It is clear that the four geometries are significantly different from each other, but the time series is very jagged. Let's use the information about the clouds:

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
#filtered = data[data.indices_CLP_mean<0.2]
filtered = data[data.masks_CLM_mean<0.4]

for idx, _type in enumerate(filtered.type.unique()):
    series = filtered[filtered.type==_type]
    series.plot(ax=ax, x='interval_from', y='indices_NDVI_mean', color=f'C{idx}', label=_type);
    ax.fill_between(series.interval_from.values, series.indices_NDVI_mean-series.indices_NDVI_stDev, series.indices_NDVI_mean+series.indices_NDVI_stDev, color=f'C{idx}', alpha=0.3 );